In [1]:
import pandas as p
train=p.read_csv("736project data.tsv", delimiter='\t')
y=train['Bottle'].values
X=train['Review'].values

# Yamazaki Sherry 2016-0
# Yamazaki 12 Year Old-1
# Nikka Whisky From The Barrel-2

# positive 0
# neutral 1
# negative 2

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=3, stop_words='english')

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,3), min_df=2, stop_words='english')

# U-Gram SVM Bottle Review feature words

In [6]:
X_vec = unigram_count_vectorizer.fit_transform(X)

In [7]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_vec,y)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
svm_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')),('svm', LinearSVC())])
scores = cross_val_score(svm_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)



0.5964342948717949


In [9]:
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])


Yamazaki Sherry Cask 2016 feature words
(0.670963034810762, 'right')
(0.6795864064814445, 'glad')
(0.7834378522315931, 'rich')
(0.8288078299007421, 'tasty')
(0.8991672490553362, 'bang')
(0.9168052598312819, 'auction')
(0.9562619495322106, 'world')
(0.985297371501863, 'mind')
(1.0298075270952967, 'superb')
(1.0560005096158611, 'job')


In [141]:
feature_ranks = sorted(zip(svm_clf.coef_[1], unigram_count_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()

Yamazaki 12 Year Old feature words
(0.6767582066066857, 'lovely')
(0.7690345988913907, 'soft')
(0.7700945811937437, 'profile')
(0.7850913188486477, 'drop')
(0.7907242668754786, 'write')
(0.81517127439993, 'smooth')
(0.8391015283302403, 'distillery')
(0.8785478975965563, 'chocolate')
(0.8985871091981449, 'overrated')
(0.8987248133705696, 'exotic')



In [142]:
feature_ranks = sorted(zip(svm_clf.coef_[2], unigram_count_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(0.6463015926149444, 'toffee')
(0.6487697474356708, 'blended')
(0.7112167711961717, 'pleasant')
(0.7393777005884247, 'drinkable')
(0.7581678998733733, 'aftertaste')
(0.7807101306263778, 'thank')
(0.7887779243763926, 'fantastic')
(0.8328913227364254, 'high')
(0.9408376614501961, 'packaging')
(0.9524326300158497, 'nikka')



In [8]:
from sklearn.model_selection import cross_validate

In [11]:
scoring = ['precision_macro', 'recall_macro']
scores = cross_validate(svm_clf, X_vec, y, scoring=scoring,cv=5, return_train_score=False)
print(scores)

{'fit_time': array([0.00400114, 0.00304508, 0.00344181, 0.00327206, 0.00323105]), 'score_time': array([0.00082493, 0.00079203, 0.00073814, 0.00107694, 0.00081611]), 'test_precision_macro': array([0.51388889, 0.67673993, 0.72646465, 0.76181167, 0.69999147]), 'test_recall_macro': array([0.47932367, 0.62391304, 0.72934783, 0.75652174, 0.67137681])}


# U-Gram SVM Bottle prediction

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])



(231,) (231,) (155,) (155,)
I enjoyed my bottle of Nikka, found it a little harsh neat, but just a drop of water opened it up nicely and made it thoroughly drinkable. 
2
The best blended I have ever tried , full (but not too full) and kind of sweet , almost perfect 
2


In [97]:
training_labels = set(y_train)
print(training_labels)
from scipy.stats import itemfreq
training_category_dist = itemfreq(y_train)
print(training_category_dist)
testing_category_dist = itemfreq(y_test)
print(testing_category_dist)

{0, 1, 2}
[[ 0 68]
 [ 1 73]
 [ 2 90]]
[[ 0 47]
 [ 1 48]
 [ 2 60]]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  


In [13]:
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
X_test_vec = unigram_count_vectorizer.transform(X_test)

In [14]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_train_vec,y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [15]:
svm_clf.score(X_test_vec,y_test)

0.6387096774193548

In [108]:
from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[27  9 11]
 [ 2 29 17]
 [ 2 15 43]]

             precision    recall  f1-score   support

          0       0.87      0.57      0.69        47
          1       0.55      0.60      0.57        48
          2       0.61      0.72      0.66        60

avg / total       0.67      0.64      0.64       155



# N-Gram SVM Bottle review feature words

In [143]:
X_vec = gram12_count_vectorizer.fit_transform(X)

In [144]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_vec,y)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [145]:
svm_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')),('svm', LinearSVC())])
scores = cross_val_score(svm_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.5964342948717949


In [146]:
feature_ranks = sorted(zip(svm_clf.coef_[0], gram12_count_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])



Yamazaki Sherry Cask 2016 feature words
(0.6040900855760505, 'far')
(0.6466787869705506, 'people')
(0.739378060695505, 'auction')
(0.7802462497862717, 'rich')
(0.8327756503634887, 'tasty')
(0.898200840199263, 'bang')
(0.9414181359251721, 'world')
(0.9855493399713362, 'mind')
(1.054678195332329, 'job')
(1.0745025304356088, 'superb')


In [147]:
feature_ranks = sorted(zip(svm_clf.coef_[1], gram12_count_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()

Yamazaki 12 Year Old feature words
(0.5910195491448946, 'smokey')
(0.6135634543173293, 'overrated')
(0.642641351442899, 'nice')
(0.6462417199281614, 'scotch')
(0.7100688887974985, 'lovely')
(0.7342092154955846, 'write')
(0.735060995718528, 'smooth')
(0.7640959709105685, 'chocolate')
(0.8193415388927124, 'soft')
(0.8335827604694461, 'distillery')



In [148]:
feature_ranks = sorted(zip(svm_clf.coef_[2], gram12_count_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(0.5316410804458506, 'offering')
(0.5336115497288053, 'spent')
(0.5778781544564764, 'value')
(0.6137407541348239, 'high')
(0.6351853952643068, 'time')
(0.6395515914640089, 'drinkable')
(0.7754994951787728, 'nikka')
(0.7767302956300499, 'fantastic')
(0.780480061512767, 'thank')
(0.8597410613123646, 'packaging')



# N-Gram SVM Bottle prediction

In [77]:
X_train_vec = gram12_count_vectorizer.fit_transform(X_train)
X_test_vec = gram12_count_vectorizer.transform(X_test)

In [78]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_train_vec,y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [79]:
svm_clf.score(X_test_vec,y_test)


0.5419354838709678

In [80]:

from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[18  1 28]
 [ 6 13 29]
 [ 2  5 53]]

             precision    recall  f1-score   support

          0       0.69      0.38      0.49        47
          1       0.68      0.27      0.39        48
          2       0.48      0.88      0.62        60

avg / total       0.61      0.54      0.51       155



# Uni-Gram MNB Bottle review feature words

In [118]:
# MNB
X_vec = unigram_count_vectorizer.fit_transform(X)

from sklearn.naive_bayes import MultinomialNB

# initialize the MNB model
nb_clf= MultinomialNB()

# use the training data to train the MNB model
nb_clf.fit(X_vec,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [119]:
nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.725801282051282


In [120]:
feature_ranks = sorted(zip(nb_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])

Yamazaki Sherry Cask 2016 feature words
(-4.9285947072168295, 'don')
(-4.892227063045954, 'just')
(-4.758695670421432, 'people')
(-4.727924011754679, 'mom')
(-4.640912634765049, 'bottles')
(-4.613513660576935, 'lottery')
(-4.3742839715111, 'price')
(-4.3742839715111, 'whisky')
(-4.292605940496832, 'auction')
(-4.146894129315439, 'bottle')


In [121]:
feature_ranks = sorted(zip(nb_clf.coef_[1], unigram_count_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()



Yamazaki 12 Year Old feature words
(-4.892738615619135, 'japanese')
(-4.892738615619135, 'malt')
(-4.848286853048301, 'smooth')
(-4.764905244109251, 'bottle')
(-4.725684530955968, 'yamazaki')
(-4.6515765588022475, 'taste')
(-4.582583687315296, '12')
(-4.582583687315296, 'price')
(-4.428433007488037, 'good')
(-4.091960770866825, 'whisky')



In [122]:
feature_ranks = sorted(zip(nb_clf.coef_[2], unigram_count_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(-5.086253512417473, 'really')
(-5.043693897998677, 'just')
(-5.002871903478422, 'taste')
(-4.8895432181714185, 'whiskey')
(-4.820550346684467, 'nose')
(-4.725240166880143, 'japanese')
(-4.695387203730461, 'good')
(-4.558186082216976, 'like')
(-4.460547612653061, 'bottle')
(-3.7674004320931154, 'whisky')



# TFIDF SVM feature words

In [123]:
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=3, stop_words='english')

In [124]:
X_vec = unigram_tfidf_vectorizer.fit_transform(X)

In [125]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_vec,y)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [126]:
svm_clf_pipe = Pipeline([('vect', TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=3, stop_words='english')),('svm', LinearSVC())])
scores = cross_val_score(svm_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.6895032051282052


In [127]:
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_tfidf_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])

Yamazaki Sherry Cask 2016 feature words
(0.9616072575140195, 'flippers')
(0.9634556062515954, 'world')
(0.9702166417795605, 'glad')
(0.9786474157787991, 'opportunity')
(1.00730160977192, 'sherry')
(1.0852120461801884, 'lottery')
(1.0886977850534687, 'job')
(1.1900341568591837, 'mom')
(1.520619153813523, 'people')
(1.829281592394503, 'auction')


In [128]:
feature_ranks = sorted(zip(svm_clf.coef_[1], unigram_tfidf_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()

Yamazaki 12 Year Old feature words
(1.0519049285092643, 'wife')
(1.0573782307173791, 'lovely')
(1.0959469733916718, 'yamazaki')
(1.1057308153744816, 'flavor')
(1.1134739015550361, 'chocolate')
(1.1260331655861036, '18')
(1.1374152136730071, 'overrated')
(1.1563677591044783, 'exotic')
(1.492611495467483, '12')
(1.6562350425106793, 'smooth')



In [129]:

feature_ranks = sorted(zip(svm_clf.coef_[2], unigram_tfidf_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(1.0629376564025141, 'water')
(1.0658971028185533, 'pleasant')
(1.176079940313953, 'drinkable')
(1.1962137513357352, 'aftertaste')
(1.2005106328232729, 'blended')
(1.2460236528392759, 'value')
(1.2665829685485617, 'toffee')
(1.2749665295864347, 'high')
(1.3344971546534008, 'packaging')
(1.4914464155730798, 'nikka')



# TFDIF Svm prediction


In [56]:
X_train_vec = unigram_tfidf_vectorizer.fit_transform(X_train)
X_test_vec = unigram_tfidf_vectorizer.transform(X_test)

In [57]:

from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_train_vec,y_train)

svm_clf.score(X_test_vec,y_test)

0.6645161290322581

In [58]:
from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[32  7  8]
 [ 1 28 19]
 [ 5 12 43]]

             precision    recall  f1-score   support

          0       0.84      0.68      0.75        47
          1       0.60      0.58      0.59        48
          2       0.61      0.72      0.66        60

avg / total       0.68      0.66      0.67       155



# Uni-gram MNB feature words

In [131]:
# MNB
X_vec = unigram_count_vectorizer.fit_transform(X)

from sklearn.naive_bayes import MultinomialNB

# initialize the MNB model
nb_clf= MultinomialNB()

# use the training data to train the MNB model
nb_clf.fit(X_vec,y)



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [132]:

nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.725801282051282


In [133]:

feature_ranks = sorted(zip(nb_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])

Yamazaki Sherry Cask 2016 feature words
(-4.9285947072168295, 'don')
(-4.892227063045954, 'just')
(-4.758695670421432, 'people')
(-4.727924011754679, 'mom')
(-4.640912634765049, 'bottles')
(-4.613513660576935, 'lottery')
(-4.3742839715111, 'price')
(-4.3742839715111, 'whisky')
(-4.292605940496832, 'auction')
(-4.146894129315439, 'bottle')


In [134]:

feature_ranks = sorted(zip(nb_clf.coef_[1], unigram_count_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()

Yamazaki 12 Year Old feature words
(-4.892738615619135, 'japanese')
(-4.892738615619135, 'malt')
(-4.848286853048301, 'smooth')
(-4.764905244109251, 'bottle')
(-4.725684530955968, 'yamazaki')
(-4.6515765588022475, 'taste')
(-4.582583687315296, '12')
(-4.582583687315296, 'price')
(-4.428433007488037, 'good')
(-4.091960770866825, 'whisky')



In [135]:
feature_ranks = sorted(zip(nb_clf.coef_[2], unigram_count_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(-5.086253512417473, 'really')
(-5.043693897998677, 'just')
(-5.002871903478422, 'taste')
(-4.8895432181714185, 'whiskey')
(-4.820550346684467, 'nose')
(-4.725240166880143, 'japanese')
(-4.695387203730461, 'good')
(-4.558186082216976, 'like')
(-4.460547612653061, 'bottle')
(-3.7674004320931154, 'whisky')



# Uni-Gram MNB Bottle prediction

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
X_test_vec = unigram_count_vectorizer.transform(X_test)

In [144]:
nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,y_train)

nb_clf.score(X_test_vec,y_test)

0.6967741935483871

In [135]:

from sklearn.metrics import confusion_matrix
y_pred = nb_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[36  7  4]
 [ 1 31 16]
 [ 5 14 41]]

             precision    recall  f1-score   support

          0       0.86      0.77      0.81        47
          1       0.60      0.65      0.62        48
          2       0.67      0.68      0.68        60

avg / total       0.70      0.70      0.70       155



# N-Gram MNB Bottle review feature words

In [25]:
X_vec = gram12_count_vectorizer.fit_transform(X)

In [26]:
from sklearn.naive_bayes import MultinomialNB
nb_clf= MultinomialNB()
nb_clf.fit(X_vec,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.725801282051282


In [127]:

feature_ranks = sorted(zip(nb_clf.coef_[0], gram12_count_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])


Yamazaki Sherry Cask 2016 feature words
(-5.160331623574134, 'don')
(-5.123963979403259, 'just')
(-4.990432586778736, 'people')
(-4.959660928111983, 'mom')
(-4.872649551122353, 'bottles')
(-4.845250576934239, 'lottery')
(-4.606020887868405, 'price')
(-4.606020887868405, 'whisky')
(-4.524342856854137, 'auction')
(-4.378631045672743, 'bottle')


In [128]:


feature_ranks = sorted(zip(nb_clf.coef_[1], gram12_count_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()



Yamazaki 12 Year Old feature words
(-5.140865790205861, 'japanese')
(-5.140865790205861, 'malt')
(-5.096414027635028, 'smooth')
(-5.0130324186959765, 'bottle')
(-4.973811705542696, 'yamazaki')
(-4.899703733388973, 'taste')
(-4.830710861902022, '12')
(-4.830710861902022, 'price')
(-4.676560182074764, 'good')
(-4.340087945453551, 'whisky')



In [61]:
feature_ranks = sorted(zip(nb_clf.coef_[2], gram12_count_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(-5.356680939574526, 'golden')
(-5.348561903188153, 'discover')
(-5.319715235834198, 'chance people')
(-5.296468036654539, 'good whisky')
(-5.218947828899452, 'couple years')
(-5.162857961433719, 'consider')
(-5.1372269457063275, 'help')
(-5.136001038021743, 'add')
(-5.113462103194222, 'aftertaste')
(-4.7047894237603956, 'hibiki 12')



# N-Gram MNB Bottle prediction

In [130]:
X_train_vec = gram12_count_vectorizer.fit_transform(X_train)
X_test_vec = gram12_count_vectorizer.transform(X_test)


In [136]:
nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,y_train)

nb_clf.score(X_test_vec,y_test)





0.6967741935483871

In [137]:
from sklearn.metrics import confusion_matrix
y_pred = nb_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[36  7  4]
 [ 1 31 16]
 [ 5 14 41]]

             precision    recall  f1-score   support

          0       0.86      0.77      0.81        47
          1       0.60      0.65      0.62        48
          2       0.67      0.68      0.68        60

avg / total       0.70      0.70      0.70       155



# TFDIF MNB feature words

In [59]:
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=3, stop_words='english')

X_vec = unigram_tfidf_vectorizer.fit_transform(X)

In [60]:
from sklearn.naive_bayes import MultinomialNB
nb_clf= MultinomialNB()
nb_clf.fit(X_vec,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [62]:
nb_clf_pipe = Pipeline([('vect', TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=3, stop_words='english')),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)


0.7076923076923077


In [63]:

feature_ranks = sorted(zip(nb_clf.coef_[0], unigram_tfidf_vectorizer.get_feature_names()))
Yamazaki_Sherry_10 = feature_ranks[-10:]
print("Yamazaki Sherry Cask 2016 feature words")
for i in range(0, len(Yamazaki_Sherry_10)):
    print(Yamazaki_Sherry_10[i])

Yamazaki Sherry Cask 2016 feature words
(-5.421407302723988, 'fair')
(-5.403252336012319, 'just')
(-5.399268021604018, 'whisky')
(-5.285864886242632, 'mom')
(-5.255863574019035, 'price')
(-5.206779624356134, 'bottles')
(-5.152278478737692, 'people')
(-5.090362051546409, 'lottery')
(-5.051471483021324, 'bottle')
(-4.844743961943499, 'auction')


In [64]:

feature_ranks = sorted(zip(nb_clf.coef_[1], unigram_tfidf_vectorizer.get_feature_names()))
Yamazaki_12_10 = feature_ranks[-10:]
print("Yamazaki 12 Year Old feature words")
for i in range(0, len(Yamazaki_12_10)):
    print(Yamazaki_12_10[i])
print()

Yamazaki 12 Year Old feature words
(-5.378406133006436, 'yamazaki')
(-5.364261173428774, 'single')
(-5.343717871375954, 'malt')
(-5.322271473070919, 'taste')
(-5.193765827331557, 'nice')
(-5.146217638964824, 'best')
(-5.124310219600333, '12')
(-5.037903928482103, 'smooth')
(-4.983048594246812, 'good')
(-4.817319101962265, 'whisky')



In [65]:

feature_ranks = sorted(zip(nb_clf.coef_[2], unigram_tfidf_vectorizer.get_feature_names()))
Nikka_10 = feature_ranks[-10:]
print("Nikka Whisky From The Barrel feature words")
for i in range(0, len(Nikka_10)):
    print(Nikka_10[i])
print()

Nikka Whisky From The Barrel feature words
(-5.356680939574526, 'taste')
(-5.348561903188153, 'nikka')
(-5.319715235834198, 'good')
(-5.296468036654539, 'time')
(-5.218947828899452, 'like')
(-5.162857961433719, 'japanese')
(-5.1372269457063275, 'whiskey')
(-5.136001038021743, 'best')
(-5.113462103194222, 'bottle')
(-4.7047894237603956, 'whisky')



# TFDIF MNB prediction

In [ ]:
X_train_vec = unigram_tfidf_vectorizer.fit_transform(X_train)
X_test_vec = unigram_tfidf_vectorizer.transform(X_test)

In [66]:
nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,y_train)

nb_clf.score(X_test_vec,y_test)

0.6258064516129033

In [67]:

from sklearn.metrics import confusion_matrix
y_pred = nb_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[31  7  9]
 [ 1 20 27]
 [ 5  9 46]]

             precision    recall  f1-score   support

          0       0.84      0.66      0.74        47
          1       0.56      0.42      0.48        48
          2       0.56      0.77      0.65        60

avg / total       0.64      0.63      0.62       155



# Uni-gram Knn feature words

In [41]:
X_vec = unigram_count_vectorizer.fit_transform(X)

from sklearn.neighbors import KNeighborsClassifier

# initialize the MNB model
neigh_clf= KNeighborsClassifier(n_neighbors=3)

# use the training data to train the MNB model
neigh_clf.fit(X_vec,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [35]:
knn_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')),('knn', KNeighborsClassifier())])
scores = cross_val_score(knn_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.42752403846153847


# Uni-gram Knn prediction

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
X_test_vec = unigram_count_vectorizer.transform(X_test)

In [37]:
neigh_clf= KNeighborsClassifier(n_neighbors=3)
neigh_clf.fit(X_train_vec,y_train)

neigh_clf.score(X_test_vec,y_test)

0.36774193548387096

In [38]:
from sklearn.metrics import confusion_matrix
y_pred = neigh_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[ 8  5 34]
 [ 6 12 30]
 [ 6 17 37]]

             precision    recall  f1-score   support

          0       0.40      0.17      0.24        47
          1       0.35      0.25      0.29        48
          2       0.37      0.62      0.46        60

avg / total       0.37      0.37      0.34       155



# TFDIF knn feature words

In [68]:
X_vec = unigram_tfidf_vectorizer.fit_transform(X)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# initialize the MNB model
neigh_clf= KNeighborsClassifier(n_neighbors=3)

# use the training data to train the MNB model
neigh_clf.fit(X_vec,y)

In [69]:

knn_clf_pipe = Pipeline([('vect', TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=3, stop_words='english')),('knn', KNeighborsClassifier())])
scores = cross_val_score(knn_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.5545272435897436


# TFDIF knn prediction

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


X_train_vec = unigram_tfidf_vectorizer.fit_transform(X_train)
X_test_vec = unigram_tfidf_vectorizer.transform(X_test)

In [71]:
neigh_clf= KNeighborsClassifier(n_neighbors=3)
neigh_clf.fit(X_train_vec,y_train)

neigh_clf.score(X_test_vec,y_test)

0.38064516129032255

In [72]:




from sklearn.metrics import confusion_matrix
y_pred = neigh_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2])
print(cm)
print()
from sklearn.metrics import classification_report
target_names = ['0','1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

[[ 0  1 46]
 [ 0  0 48]
 [ 0  1 59]]

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        47
          1       0.00      0.00      0.00        48
          2       0.39      0.98      0.55        60

avg / total       0.15      0.38      0.21       155



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
